# Set up

In [13]:
import os
import json
import yaml
import shutil
import zipfile
import random
import numpy as np
import pandas as pd

# Import labels

In [2]:
os.getcwd()

'/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code'

In [3]:
!ls ../data_atl

img_atl_2022  img_middown_0206	inference_log  seg
img_cs_0204   img_middown_0208	inferred


In [4]:
base_dir = '../data_atl'

In [5]:
base_dir = '../data_atl'
folder_path = os.path.join(base_dir, "inferred", "inferred_model_nc_5")
print(folder_path)
print()
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
filenames = sorted(filenames)
filenames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
print(len(filenames))
filenames[:5]

# {edge_id}__{point_id}__{side}__{j}__{class_id}__{confidence}__{area}

../data_atl/inferred/inferred_model_nc_5

406


['../data_atl/inferred/inferred_model_nc_5/224__1__left__0__2__0.29__714.jpg',
 '../data_atl/inferred/inferred_model_nc_5/458__13__right__0__1__0.3__2184.jpg',
 '../data_atl/inferred/inferred_model_nc_5/531__5__left__0__4__0.61__420.jpg',
 '../data_atl/inferred/inferred_model_nc_5/210__13__right__0__1__0.22__650.jpg',
 '../data_atl/inferred/inferred_model_nc_5/368__12__left__0__3__0.24__713.jpg']

In [6]:
filenames = sorted(filenames)
filenames

['../data_atl/inferred/inferred_model_nc_5/100__15__right__1__4__0.11__630.jpg',
 '../data_atl/inferred/inferred_model_nc_5/100__17__right__0__0__0.51__435.jpg',
 '../data_atl/inferred/inferred_model_nc_5/101__0__right__0__3__0.25__1260.jpg',
 '../data_atl/inferred/inferred_model_nc_5/102__3__left__0__3__0.47__418.jpg',
 '../data_atl/inferred/inferred_model_nc_5/103__8__left__0__3__0.26__425.jpg',
 '../data_atl/inferred/inferred_model_nc_5/106__14__left__0__4__0.41__500.jpg',
 '../data_atl/inferred/inferred_model_nc_5/109__0__left__0__3__0.71__816.jpg',
 '../data_atl/inferred/inferred_model_nc_5/109__0__left__1__2__0.16__782.jpg',
 '../data_atl/inferred/inferred_model_nc_5/109__0__right__0__4__0.12__1323.jpg',
 '../data_atl/inferred/inferred_model_nc_5/111__1__right__0__2__0.21__1075.jpg',
 '../data_atl/inferred/inferred_model_nc_5/111__6__right__0__1__0.17__432.jpg',
 '../data_atl/inferred/inferred_model_nc_5/113__5__left__0__1__0.14__748.jpg',
 '../data_atl/inferred/inferred_model_nc

In [56]:
base_dir = '../data_atl'
log_detected_df = pd.read_csv(os.path.join(base_dir, "inferred", "log_detected_df.csv")).drop(columns = ["Unnamed: 0"])
log_detected_df = log_detected_df.sort_values(by="saved_filename")
log_detected_df

,range_index,file_id,detection_index,class_id,xmin,ymin,xmax,ymax,confidence,area,saved_filename
185,33,100__15__right,1,4,0,0,15,42,0.11,630,100__15__right__1__4__0.11__630.jpg
214,37,100__17__right,0,0,593,293,608,322,0.51,435,100__17__right__0__0__0.51__435.jpg
217,38,101__0__right,0,3,349,291,377,336,0.25,1260,101__0__right__0__3__0.25__1260.jpg
404,77,102__3__left,0,3,238,281,257,303,0.47,418,102__3__left__0__3__0.47__418.jpg
325,58,103__8__left,0,3,144,306,161,331,0.26,425,103__8__left__0__3__0.26__425.jpg
...,...,...,...,...,...,...,...,...,...,...,...
161,30,94__10__left,1,1,604,339,624,363,0.12,480,94__10__left__1__1__0.12__480.jpg
87,16,94__17__right,0,3,606,267,640,334,0.21,2278,94__17__right__0__3__0.21__2278.jpg
42,7,94__2__right,0,3,217,297,238,328,0.40,651,94__2__right__0__3__0.4__651.jpg
43,7,94__2__right,1,1,217,297,237,328,0.25,620,94__2__right__1__1__0.25__620.jpg


In [57]:
class_id_nm = pd.DataFrame({
    "class_id": [0, 1, 2, 3, 4],
    "class_nm": ["prohibAnyT", "prohibSomeT", "prohibBusStop", "prohibClean", "allow"]
})
class_id_nm

,class_id,class_nm
0,0,prohibAnyT
1,1,prohibSomeT
2,2,prohibBusStop
3,3,prohibClean
4,4,allow


In [58]:
log_detected_df = log_detected_df.merge(class_id_nm, on="class_id", how="left")
log_detected_df

,range_index,file_id,detection_index,class_id,xmin,ymin,xmax,ymax,confidence,area,saved_filename,class_nm
0,33,100__15__right,1,4,0,0,15,42,0.11,630,100__15__right__1__4__0.11__630.jpg,allow
1,37,100__17__right,0,0,593,293,608,322,0.51,435,100__17__right__0__0__0.51__435.jpg,prohibAnyT
2,38,101__0__right,0,3,349,291,377,336,0.25,1260,101__0__right__0__3__0.25__1260.jpg,prohibClean
3,77,102__3__left,0,3,238,281,257,303,0.47,418,102__3__left__0__3__0.47__418.jpg,prohibClean
4,58,103__8__left,0,3,144,306,161,331,0.26,425,103__8__left__0__3__0.26__425.jpg,prohibClean
...,...,...,...,...,...,...,...,...,...,...,...,...
401,30,94__10__left,1,1,604,339,624,363,0.12,480,94__10__left__1__1__0.12__480.jpg,prohibSomeT
402,16,94__17__right,0,3,606,267,640,334,0.21,2278,94__17__right__0__3__0.21__2278.jpg,prohibClean
403,7,94__2__right,0,3,217,297,238,328,0.40,651,94__2__right__0__3__0.4__651.jpg,prohibClean
404,7,94__2__right,1,1,217,297,237,328,0.25,620,94__2__right__1__1__0.25__620.jpg,prohibSomeT


In [59]:
log_detected_df.columns

Index(['range_index', 'file_id', 'detection_index', 'class_id', 'xmin', 'ymin',
       'xmax', 'ymax', 'confidence', 'area', 'saved_filename', 'class_nm'],
      dtype='object')

# Build json file 

## extract category from filename

In [81]:
# Create the COCO dictionary
coco_format = {
    "images": [],
    "annotations": [],
    "categories": []
}

# Extract unique image files and assign image IDs
image_files = log_detected_df["saved_filename"].unique()
image_id_map = {file: i for i, file in enumerate(image_files)}
#print(image_id_map)
#print() 

# Extract unique class names and assign category IDs
class_names = log_detected_df["class_nm"].unique()
new_order = ['prohibAnyT', 'prohibSomeT', 'prohibBusStop','prohibClean', 'allow']
class_names = np.array(new_order)
category_id_map = {name: i for i, name in enumerate(class_names, 1)} # very important to start from 1 not 0 for the category_id
print(category_id_map)

# Add category information
for class_name, class_id in category_id_map.items():
    coco_format["categories"].append({
        "id": class_id,
        "name": class_name,
        #"supercategory": "sign"
    })

# Add image metadata
for file_id in image_id_map:
    image_rows = log_detected_df[log_detected_df["saved_filename"] == file_id]
    #print(image_rows)
    # Assuming all detections for the same file share the same dimensions
    coco_format["images"].append({
        "id": image_id_map[file_id],
        "file_name": str(image_rows["saved_filename"].iloc[0]),  # very important
        "width": 640,
        "height": 640
    })
    
# Add annotation data
annotation_id = 0
for _, row in log_detected_df.iterrows():
    coco_format["annotations"].append({
        "id": annotation_id,
        "image_id": image_id_map[row["saved_filename"]],   # HERE
        "category_id": category_id_map[row["class_nm"]],
        "bbox": [row["xmin"], row["ymin"], row["xmax"] - row["xmin"], row["ymax"] - row["ymin"]],
        "area": row["area"],
        "iscrowd": 0,
        #"confidence": row["confidence"]
        "segmentation": []
    })
    annotation_id += 1

{'prohibAnyT': 1, 'prohibSomeT': 2, 'prohibBusStop': 3, 'prohibClean': 4, 'allow': 5}


In [82]:
coco_format

{'images': [{'id': 0,
   'file_name': '100__15__right__1__4__0.11__630.jpg',
   'width': 640,
   'height': 640},
  {'id': 1,
   'file_name': '100__17__right__0__0__0.51__435.jpg',
   'width': 640,
   'height': 640},
  {'id': 2,
   'file_name': '101__0__right__0__3__0.25__1260.jpg',
   'width': 640,
   'height': 640},
  {'id': 3,
   'file_name': '102__3__left__0__3__0.47__418.jpg',
   'width': 640,
   'height': 640},
  {'id': 4,
   'file_name': '103__8__left__0__3__0.26__425.jpg',
   'width': 640,
   'height': 640},
  {'id': 5,
   'file_name': '106__14__left__0__4__0.41__500.jpg',
   'width': 640,
   'height': 640},
  {'id': 6,
   'file_name': '109__0__left__0__3__0.71__816.jpg',
   'width': 640,
   'height': 640},
  {'id': 7,
   'file_name': '109__0__left__1__2__0.16__782.jpg',
   'width': 640,
   'height': 640},
  {'id': 8,
   'file_name': '109__0__right__0__4__0.12__1323.jpg',
   'width': 640,
   'height': 640},
  {'id': 9,
   'file_name': '111__1__right__0__2__0.21__1075.jpg',
   'w

## export

In [83]:
base_dir = '../data_atl'
output_path = os.path.join(base_dir, "inferred", "inferred_model_nc_5.json")

In [84]:
# Save COCO JSON to file
with open(output_path, "w") as f:
    json.dump(coco_format, f, indent=4
              #, default=convert_numpy
             )

print(f"COCO annotations saved to {output_path}!")

COCO annotations saved to ../data_atl/inferred/inferred_model_nc_5.json!


# Bring in original images

In [96]:
base_dir = '../data_atl'
folder_path = os.path.join(base_dir, "img_middown_0208")
print(folder_path)
print()
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
filenames = sorted(filenames)
filenames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) & f.endswith((".jpg", ".png"))]
print(len(filenames))
filenames[:5]

# {edge_id}__{point_id}__{side}__{j}__{class_id}__{confidence}__{area}

../data_atl/img_middown_0208

7788


['../data_atl/img_middown_0208/gsv__215__12__right__139.361__2019-10__33.775806801,-84.379175182.jpg',
 '../data_atl/img_middown_0208/gsv__173__11__right__48.943__2022-03__33.785063793,-84.384255698.jpg',
 '../data_atl/img_middown_0208/gsv__530__3__left__78.461__2024-07__33.750691195,-84.393234343.jpg',
 '../data_atl/img_middown_0208/gsv__239__4__right__139.456__2024-11__33.796786698,-84.38753574.jpg',
 '../data_atl/img_middown_0208/gsv__405__9__right__168.371__2024-06__33.752104796,-84.395139153.jpg']

In [98]:
unique_detected_ids = log_detected_df["saved_filename"].tolist()
unique_detected_ids = [f.split("__") for f in unique_detected_ids] 
unique_detected_ids = ["__".join([f[0], f[1], f[2]]) for f in unique_detected_ids] 
print(len(unique_detected_ids))
unique_detected_ids[:5]

406


['100__15__right',
 '100__17__right',
 '101__0__right',
 '102__3__left',
 '103__8__left']

In [100]:
filename_of_detected = list()

for filename in filenames:
    temp = filename.split("/")[-1].split("__")
    temp = "__".join([temp[1], temp[2], temp[3]]) 
    #print(temp)
    if temp in unique_detected_ids:
        filename_of_detected.append(filename)
print(len(filename_of_detected))
filename_of_detected[:5]

357


['../data_atl/img_middown_0208/gsv__413__12__right__nan__2024-07__33.751050768,-84.393232338.jpg',
 '../data_atl/img_middown_0208/gsv__503__13__left__-56.362__2024-11__33.757213141,-84.381455751.jpg',
 '../data_atl/img_middown_0208/gsv__468__5__left__135.041__2024-07__33.754496624,-84.387807485.jpg',
 '../data_atl/img_middown_0208/gsv__390__9__left__78.267__2024-07__33.750005066,-84.391331876.jpg',
 '../data_atl/img_middown_0208/gsv__76__4__left__140.012__2023-01__33.772343059,-84.387770359.jpg']

# HERE
merge "filename_of_detected" to the "log_detected_df" to make the annotation.json again  

In [103]:
filename_of_detected = [f.split("/")[-1] for f in filename_of_detected]
print(len(filename_of_detected))
filename_of_detected[:5]

357


['gsv__413__12__right__nan__2024-07__33.751050768,-84.393232338.jpg',
 'gsv__503__13__left__-56.362__2024-11__33.757213141,-84.381455751.jpg',
 'gsv__468__5__left__135.041__2024-07__33.754496624,-84.387807485.jpg',
 'gsv__390__9__left__78.267__2024-07__33.750005066,-84.391331876.jpg',
 'gsv__76__4__left__140.012__2023-01__33.772343059,-84.387770359.jpg']

# Disregard

In [67]:
# Custom function to convert numpy types to Python native types
def convert_numpy(obj):
    if isinstance(obj, np.integer):  # Convert np.int64, np.int32, etc.
        return int(obj)
    if isinstance(obj, np.floating):  # Convert np.float64, np.float32, etc.
        return float(obj)
    return obj  # Default case



COCO annotations saved to ../data_atl/inferred/inferred_model_nc_5.json!


In [8]:
# Define function to extract category from filename
def extract_category(filename):
    category = filename.split("/")[-1].split("__")[4]
    return category

In [9]:
extract_category(filenames[0])

'4'

In [19]:
def build_coco_annotation(row):



In [42]:
# Generate JSON for CVAT auto-labeling
base_dir = '../data_atl'
output_path = os.path.join(base_dir, "inferred", "inferred_model_nc_5.json")

coco_data = {
    "categories": [],
    "images": [],
    "annotations": []
}

# Map for unique category IDs
category_map = {}
category_id = 1  # Start category IDs at 1
annotation_id = 1  # Start annotation IDs at 1

# Process each filename to populate the COCO structure
for i in range(len(log_detected_df)):
    # Extract category
    row = log_detected_df.iloc[i] 
    
    class_nm = row["class_nm"]
    class_id = row["class_id"]
    xmin = row["xmin"]
    ymin = row["ymin"]
    xmax = row["xmax"]
    ymax = row["ymax"]

    # Add category to the categories list if not already added
    if class_nm not in category_map:
        category_map[class_nm] = class_id
        coco_data["categories"].append({
            "id": class_id,
            "name": class_nm
        })
        class_id += 1

    # Add image metadata
    coco_data["images"].append({
        "id": image_id,
        "file_name": filename,
        "width": 640,  # Placeholder (set actual width if available)
        "height": 640  # Placeholder (set actual height if available)
    })

    # Add empty annotation (bounding box is blank)
    coco_data["annotations"].append({
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_map[category_name],
        "bbox": [100,100,300,300],  # Empty bounding box
        "area": 0,  # Placeholder (area calculation requires bbox)
        "iscrowd": 0,  # Default to 0 (not crowded)
        "segmentation": [] # empty segmentation
    })
    annotation_id += 1

coco_data

NameError: name 'image_id' is not defined

In [ ]:
# Save COCO JSON to file
with open(output_path, "w") as f:
    json.dump(coco_data, f, indent=4)

print(f"COCO annotations saved to {output_path}!")

# Preload known annotations 